In [68]:
import os
import time
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

%matplotlib inline

In [69]:
video_dir = '/mnt/disks/datastorage/videos/'
weights_dir = '/mnt/disks/datastorage/weights/'

video_path = os.path.join(video_dir,'ballroom','v_Shooting_g30_v1_c.MP4')
flownet_weights = os.path.join(weights_dir, 'FlowNet2-CSS.pth.tar')
spatial_weights = os.path.join('/home/mlp/two-stream-action-recognition/record/spatial', 'model_best.pth.tar')
motion_weights = os.path.join('/home/mlp/two-stream-action-recognition/record/motion', 'model_best.pth.tar')
flownet_model = 'FlowNet2CSS'
nb_classes = 3
skip_num = 1

In [73]:
!python pipeline.py --stream $video_path \
                    --nb_classes $nb_classes \
                    --model $flownet_model \
                    --optical_weights $flownet_weights \
                    --spatial_weights $spatial_weights \
                    --motion_weights $motion_weights

Parsing Arguments
  [0.005s] fp16: False
  [0.005s] fp16_scale: 1024.0
  [0.005s] image_size: [224, 224]
  [0.005s] images: []
  [0.005s] inference_size: [-1, -1]
  [0.005s] loss: L1Loss
  [0.005s] model: FlowNet2CSS
  [0.005s] model_batchNorm: False
  [0.005s] model_div_flow: 20.0
  [0.005s] motion_weights: /home/mlp/two-stream-action-recognition/record/motion/model_best.pth.tar
  [0.005s] nb_classes: 3
  [0.005s] number_gpus: 1
  [0.005s] optical_weights: /mnt/disks/datastorage/weights/FlowNet2-CSS.pth.tar
  [0.005s] rgb_max: 255.0
  [0.005s] seed: 1
  [0.005s] skip_frames: 1
  [0.005s] spatial_weights: /home/mlp/two-stream-action-recognition/record/spatial/model_best.pth.tar
  [0.005s] stream: /mnt/disks/datastorage/videos/ballroom/v_Shooting_g30_v1_c.MP4
  [0.005s] Operation finished

Building FlowNet2CSS model
/home/mlp/LENS/flownet2-pytorch/models.py:460: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  init.uniform(m.bias)
/home/mlp/LENS/flownet2-py

In [ ]:
video_dir = os.path.dirname(os.path.abspath(video_path))
video_name = os.path.splitext(os.path.basename(video_path))[0]
pickle_path = os.path.join(video_dir, '{}_predictions.pkl'.format(video_name))

with open(pickle_path, 'rb') as pf:
    predictions = pickle.load(pf)
    predictions = np.array(predictions).squeeze()
    actions = np.argmax(predictions, axis=1)

In [ ]:
#for i in predictions:
#    print(i)

In [ ]:
action_dict = {
    0: 'Theft',
    1: 'Assault',
    2: 'Shooting',
    3: 'No Action',
}

cap = cv2.VideoCapture(video_path)

frame_time = 0
frame_idx = 0
display = None
i = 0
while True:
    start_time = time.time()
    
    ret, frame = cap.read()
    if not ret:
        break
    
    try:
        label = action_dict[actions[frame_idx]]
    except IndexError:
        break
        
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    cv2.putText(
        frame,
        label,
        (10, 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        2,
        (255, 255, 255),
        thickness=3
    )
    
    if i < 50:
        frame_time += time.time() - start_time
        frame_idx += 1
        i+=1
        continue
        
        
    else:
        plt.axis('off')
        plt.imshow(frame)
        plt.show()
        clear_output(wait=True)

        frame_time += time.time() - start_time
        frame_idx += 1
cap.release()

frame_time /= frame_idx
print('Avg. frame processing time: {} s'.format(round(frame_time, 2)))